In [1]:
!pip install sagemaker --upgrade
!pip install boto3 --upgrade
!pip install botocore --upgrade
# '2.203.1', '1.34.19', '1.34.19'

In [22]:
import sagemaker
import boto3
import botocore
sagemaker.__version__, boto3.__version__, botocore.__version__
# ('2.203.1', '1.34.19', '1.34.19')

('2.203.1', '1.34.20', '1.34.20')

In [23]:
import sagemaker
from sagemaker.serializers import CSVSerializer, JSONSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer
from sagemaker.explainer import (
    ExplainerConfig, 
    ClarifyExplainerConfig, 
    ClarifyShapConfig, 
    ClarifyInferenceConfig, 
    ClarifyShapBaselineConfig, 
    ClarifyTextConfig
)

json_serializer = JSONSerializer()
json_deserializer = JSONDeserializer()
csv_serializer = CSVSerializer()
csv_deserializer = CSVDeserializer()
shap_baseline = csv_serializer.serialize([["<UNK>"]])

explainer_config_dict = {
    'ClarifyExplainerConfig': {
        'InferenceConfig': {
            'LabelIndex': 0,
            'ProbabilityIndex': 1,
            'FeatureTypes': ['text'],
            'FeatureHeaders': ['ItemDescription'],
        },
        'ShapConfig': {
            'ShapBaselineConfig': {
                'MimeType': 'text/csv',
                'ShapBaseline': '<UNK>'
            },
            'NumberOfSamples': 5,
            'TextConfig': {
                'Language': 'en', 
                'Granularity': 'sentence'
            }
        }
    }
}
explainer_config_dict

{'ClarifyExplainerConfig': {'InferenceConfig': {'LabelIndex': 0,
   'ProbabilityIndex': 1,
   'FeatureTypes': ['text'],
   'FeatureHeaders': ['ItemDescription']},
  'ShapConfig': {'ShapBaselineConfig': {'MimeType': 'text/csv',
    'ShapBaseline': '<UNK>'},
   'NumberOfSamples': 5,
   'TextConfig': {'Language': 'en', 'Granularity': 'sentence'}}}}

In [25]:
import boto3
boto3_session = boto3.session.Session()
sagemaker_client = boto3.client("sagemaker")

sagemaker_runtime_client = boto3.client("sagemaker-runtime")
sagemaker_session = sagemaker.Session(
    boto_session=boto3_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=sagemaker_runtime_client,
)

version = "application-csv-multiple-items-2"
endpoint_name = f'nlp-ole-example-with-clarify-{version}'
# sagemaker_client.delete_model(ModelName=endpoint_name)    
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)



In [27]:
from datetime import datetime
import time

import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role


print("Current Date and Time:", datetime.now())

role = get_execution_role()
image_uri = f'123456789012.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-studio-d-ebafjgrwlbqa:application-csv-multiple-items'

start = time.time()

response = sagemaker_client.create_model(
    ModelName = endpoint_name,
    ExecutionRoleArn = role,
    Containers = [{
        "Image": image_uri,
        "Mode": "SingleModel",
    }]
)

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_name,
    ProductionVariants=[
        {
            "VariantName": "TestVariantName",
            "ModelName": endpoint_name,
            "InitialInstanceCount": 1,
            "InstanceType": 'ml.m4.xlarge',
        }
    ],
    ExplainerConfig=explainer_config_dict,
)

response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_name,
)

sagemaker_session.wait_for_endpoint(endpoint_name)

print("Deployed after mins:", int(time.time() - start) / 60.0)
response

Current Date and Time: 2024-01-17 16:03:41.688338
----!Deployed after mins: 2.533333333333333


{'EndpointArn': 'arn:aws:sagemaker:eu-west-1:123456789012:endpoint/nlp-ole-example-with-clarify-application-csv-multiple-items-2',
 'ResponseMetadata': {'RequestId': 'f91f91ee-fb20-41ed-bcc0-bb4f656e1a6a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f91f91ee-fb20-41ed-bcc0-bb4f656e1a6a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '129',
   'date': 'Wed, 17 Jan 2024 16:03:43 GMT'},
  'RetryAttempts': 0}}

In [28]:
sagemaker_client.describe_endpoint(EndpointName=endpoint_name)["ExplainerConfig"]

{'ClarifyExplainerConfig': {'InferenceConfig': {'ProbabilityIndex': 1,
   'LabelIndex': 0,
   'FeatureHeaders': ['ItemDescription'],
   'FeatureTypes': ['text']},
  'ShapConfig': {'ShapBaselineConfig': {'MimeType': 'text/csv',
    'ShapBaseline': '<UNK>'},
   'NumberOfSamples': 5,
   'TextConfig': {'Language': 'en', 'Granularity': 'sentence'}}}}

In [42]:
import json
import boto3
import pprint
boto3_session = boto3.session.Session()
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

data = csv_serializer.serialize("hello\nworld")

print(f"Calling: {endpoint_name}")

response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",
    Accept="text/csv",
    Body=data,
    # EnableExplanations="`false`", # "`false`" or "`true`"
)

result = json_deserializer.deserialize(response["Body"], content_type=response["ContentType"])
pprint.pprint(result)

Calling: nlp-ole-example-with-clarify-application-csv-multiple-items-2
{'explanations': {'kernel_shap': [[{'attributions': [{'attribution': [-0.37110010997317777],
                                                      'description': {'partial_text': 'hello',
                                                                      'start_idx': 0}}],
                                    'feature_header': 'ItemDescription',
                                    'feature_type': 'text'}],
                                  [{'attributions': [{'attribution': [-0.3642261388073069],
                                                      'description': {'partial_text': 'world',
                                                                      'start_idx': 0}}],
                                    'feature_header': 'ItemDescription',
                                    'feature_type': 'text'}]]},
 'label_headers': ['auto-generated-description-for-hello'],
 'predictions': {'content_type': 'text/csv; 